# Russian Synodal Bible (1885)

## Import

### Libraries

In [1]:
#import os
#from lxml import etree
from backend import *
import matplotlib
import string

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [3]:
bibleTXT = './texts/bible/sinodalnyi-perevod.txt' 
booksJSON = './texts/bible/booksDict.json'
bibleJSON = './texts/bible/bible.json'
bibleIdJSON = './texts/bible/bibleID.json'
bibleXML = './texts/bible/bible.xml'
bibFrazyTXT = './texts/bible/Dubrovina_Slovar_Bibleyskikh_Frazeologizmov.txt'

bibleOHCO = ['test', 'book', 'chap', 'verse']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'
matt_name = 'От Матфея святое благовествование'

In [4]:
with open(booksJSON) as json_file: 
    booksDict = json.load(json_file)

## Pre-Processing

### Import Text into DF

In [135]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

,text
0,\n
1,== Бытие ==\n
2,\n
3,=== 1 ===\n
4,\n
...,...
41374,17 И Дух и невеста говорят: прииди! И слышавши...
41375,18 И я также свидетельствую всякому слышащему ...
41376,19 и если кто отнимет что от слов книги пророч...
41377,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Tidy Up

In [136]:
# remove blank lines and select puncutation, keeping only header & clean text lines
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '']
bibliiaDf

/tmp/ipykernel_242246/881465790.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
/tmp/ipykernel_242246/881465790.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')


,text
1,== Бытие ==
3,=== 1 ===
5,1 В начале сотворил Бог небо и землю.
6,"2 Земля же была безвидна и пуста, и тьма над б..."
7,3 И сказал Бог: да будет свет. И стал свет.
...,...
41374,17 И Дух и невеста говорят: прииди! И слышавши...
41375,18 И я также свидетельствую всякому слышащему ...
41376,19 и если кто отнимет что от слов книги пророч...
41377,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Find Parts

In [141]:
# find df rows containing all the book and chapter title lines 
# as well as the OT/NT split on the 1st page of Matthew
chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)].index
book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)].index
test_line = bibliiaDf.loc[bibliiaDf.text.str.contains(matt_name)].index

# combine the title lines for extraction later
title_lines = test_line.append([chap_lines, book_lines]).sort_values()
title_lines

/tmp/ipykernel_242246/3255205770.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)].index
/tmp/ipykernel_242246/3255205770.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)].index


Int64Index([30299], dtype='int64', name='v_id')

### Assign OHCO Labels

In [142]:
# ['test']
bibliiaDf[bibleOHCO[0]] = np.where(bibliiaDf.index<test_line[0], 'O', 'N')
# ['book']
bibliiaDf[bibleOHCO[1]] = bibliiaDf.loc[book_lines].text.str.extract(book_lines_re)
# ['chap']
bibliiaDf[bibleOHCO[2]] = bibliiaDf.loc[chap_lines].text.str.extract(chap_lines_re)
# ['verse'] by splitting verse num at beginning from text in current verse lines
bibliiaDf[[bibleOHCO[3],'text']] = bibliiaDf.text.str.split(' ', 1, expand=True).iloc[:, [0, 1]]
# fill in book and chapter titles to cells below them
bibliiaDf[bibleOHCO[1:3]] = bibliiaDf[['book','chap']].ffill()
# drop title lines and reset index to give verse id num 
bibliiaDf = bibliiaDf.drop(title_lines, axis=0).reset_index()#.set_index(bibleOHCO)
bibliiaDf.index = range(1,len(bibliiaDf)+1)
bibliiaDf.index.name = 'v_id'
# reorder columns
bibliiaDf = bibliiaDf[['test', 'book', 'chap', 'verse', 'text']]
bibliiaDf

,test,book,chap,verse,text
v_id,,,,,
1,O,NaN,NaN,Бытие,==
2,O,NaN,NaN,1,===
3,O,NaN,NaN,В,начале сотворил Бог небо и землю.
4,O,NaN,NaN,Земля,"же была безвидна и пуста, и тьма над бездною, ..."
5,O,NaN,NaN,И,сказал Бог: да будет свет. И стал свет.
...,...,...,...,...,...
38543,N,NaN,NaN,И,Дух и невеста говорят: прииди! И слышавший да ...
38544,N,NaN,NaN,И,я также свидетельствую всякому слышащему слова...
38545,N,NaN,NaN,и,если кто отнимет что от слов книги пророчества...


### Swap Full Book Name for Abbreviation

In [143]:
book_nums = [(x+1,y) for x,y in enumerate(bibliiaDf.book.unique())]

In [ ]:
bibliiaDf.book = [booksDict[str(x)]['eng_abbr'] for x,y in book_nums for name in bibliiaDf.book if y == name]
bibliiaDf.sample(10)

### Export to JSON

## Processing

### OHCO DFs

#### Testaments

#### Books

#### Chapters

### Make XML

## Token DFs

In [99]:
TokenDf = pd.read_pickle('./proc/BibleVerseTokenDf.pkl')
TokenDf['stopword'] = ~TokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDf = TokenDf.set_index(['p_id', 'token_id'])
TokenDf

start stop      text head_id        rel    pos      lemma  \
p_id  token_id                                                             
1     1_1          0    1         В     1_2       case    ADP          в   
      1_2          2    8    начале     1_3        obl   NOUN     начало   
      1_3          9   17  сотворил     1_0       root   VERB  сотворить   
      1_4         18   21       Бог     1_3      nsubj  PROPN        бог   
      1_5         22   26      небо     1_3        obj   NOUN       небо   
...              ...  ...       ...     ...        ...    ...        ...   
37110 1_5         32   38    Христа     1_4  flat:name  PROPN    христос   
      1_6         39   41        со     1_8       case    ADP          с   
      1_7         42   47     всеми     1_8        det    DET       весь   
      1_8         48   52      вами     1_4       nmod   PRON         вы   
      2_1         54   59     Аминь     2_0       root  PROPN      аминь   

                anim aspect  case degree gender  mood number person tense  \
p_id  token_id                                                              
1     1_1       None   None  None   None   None  None   None   None  None   
      1_2       Inan   None   Loc   None   Neut  None   Sing   None  None   
      1_3       None   Perf  None   None   Masc   Ind   Sing   None  Past   
      1_4       Anim   None   Nom   None   Masc  None   Sing   None  None   
      1_5       Inan   None   Acc   None   Neut  None   Sing   None  None   
...              ...    ...   ...    ...    ...   ...    ...    ...   ...   
37110 1_5       Anim   None   Gen   None   Masc  None   Sing   None  None   
      1_6       None   None  None   None   None  None   None   None  None   
      1_7       None   None   Ins   None   None  None   Plur   None  None   
      1_8       None   None   Ins   None   None  None   Plur      2  None   
      2_1       Inan   None   Gen   None    Fem  None   Sing   None  None   

               verb_form voice  stopword  
p_id  token_id                            
1     1_1           None  None      True  
      1_2           None  None     False  
      1_3            Fin   Act     False  
      1_4           None  None     False  
      1_5           None  None     False  
...                  ...   ...       ...  
37110 1_5           None  None     False  
      1_6           None  None      True  
      1_7           None  None     False  
      1_8           None  None     False  
      2_1           None  None     False  

[676266 rows x 19 columns]

In [114]:
TokenDf.loc[TokenDf.stopword == True].lemma.value_counts()[:25]

и        57443
в        18350
не       11883
на        8968
с         6184
от        5042
к         4863
что       4792
из        4544
как       3659
а         3565
по        3089
ибо       2934
но        2905
же        2831
у         2714
за        2455
о         2444
для       2389
когда     2203
если      1851
вот       1731
то        1651
чтобы     1548
его       1486
Name: lemma, dtype: int64

In [54]:
TokenDf.loc[TokenDf[tokenCols[9]] == 'Anim'].lemma.value_counts().to_frame()

,lemma
господь,6865
сын,4698
бог,4330
царь,3216
человек,2310
...,...
гацорфия,1
цалафа,1
конский,1
фекоец,1


In [ ]:
GetRankDf(TokenDf, 'lemma')#.plot(x='rank')

,n,rank
lemma,,
он,19656,1
они,12325,2
быть,10839,3
я,10762,4
ты,7851,5
...,...,...
таинница,1,20821
избирательница,1,20822
добивать,1,20823


In [ ]:
GetRankDf(TokenDf, 'pos')

,n,rank
pos,,
NOUN,147028,1
VERB,108063,2
PRON,78480,3
ADP,70754,4
CCONJ,63398,5
PROPN,55336,6
ADJ,35327,7
DET,34131,8
PART,23999,9


In [ ]:
GetRankDf(TokenDf, 'rel')

,n,rank
rel,,
case,71010,1
nsubj,63995,2
cc,63839,3
conj,61390,4
obl,56899,5
obj,53924,6
nmod,44289,7
advmod,42893,8
root,42520,9


In [ ]:
GetRankDf(TokenDf, 'anim')

,n,rank
anim,,
Inan,145289,1
Anim,83455,2


In [ ]:
GetRankDf(TokenDf, 'person')

,n,rank
person,,
3,57935,1
2,22086,2
1,19806,3


In [ ]:
VocabDf = TokenDf.loc[~TokenDf[tokenCols[9:]].isna().all(1)].lemma.value_counts().to_frame().reset_index()#.reset_index()
VocabDf.columns = ['lemma', 'count']
VocabDf.index = VocabDf.index.map(lambda x: x + 1).rename('rank')
VocabDf.iloc[:25]#.plot(x='rank')

,lemma,count
rank,,
1,он,19656
2,они,12325
3,быть,10839
4,я,10762
5,ты,7851
6,господь,7101
7,весь,6982
8,свой,6038
9,твой,5958


In [ ]:
# TF-IDF

vocab = TokenDf.loc[TokenDf['p_id'] <= 10, 'lemma'].unique().tolist()
tokens = TokenDf.loc[TokenDf['p_id'] <= 10, 'lemma'].tolist()
total_words = len(tokens)
instances = TokenDf.loc[:, 'lemma'].value_counts().to_frame().loc['он'].lemma

num_docs = TokenDf.p_id.unique().size

TF = instances / total_words
IDF = num_docs / docsw_word

TFIDF = TF * IDF

In [16]:
bibliiaType = {'test':str, 'book':str, 'chap':int, 'verse':int}
bibliiaDf = pd.read_json(bibleJSON, orient='index', dtype=bibliiaType)
bibliiaDf.index.name = 'p_id'
#bibliiaDf.verse = bibliiaDf.verse.astype(int)
bibliiaDf

,test,book,chap,verse,text
p_id,,,,,
1,O,Gen,1,1,В начале сотворил Бог небо и землю.
2,O,Gen,1,2,"Земля же была безвидна и пуста, и тьма над без..."
3,O,Gen,1,3,И сказал Бог: да будет свет. И стал свет.
4,O,Gen,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
5,O,Gen,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37106,N,Rev,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37107,N,Rev,22,18,И я также свидетельствую всякому слышащему сло...
37108,N,Rev,22,19,и если кто отнимет что от слов книги пророчест...


In [12]:
# {p_id: {'test': }}
p_idDict = bibliiaDf.drop('text', axis=1).to_dict(orient='index')

In [13]:
PartLineNumsDict = {}
for test_name in bibliiaDf.test.unique():
    t_min = bibliiaDf.loc[bibliiaDf['test'] == test_name].reset_index().p_id.min()
    t_max = bibliiaDf.loc[bibliiaDf['test'] == test_name].reset_index().p_id.max()
    print(f"{test_name}: {t_min} - {t_max}")
    for book_name in bibliiaDf.book.unique():
        b_min = bibliiaDf.loc[bibliiaDf['book'] == book_name].reset_index().p_id.min()
        b_max = bibliiaDf.loc[bibliiaDf['book'] == book_name].reset_index().p_id.max()
        PartLineNumsDict.update({book_name: (b_min, b_max)})
        #print(f"{book_name}: {b_min} - {b_max}")
        #for chap_name in bibliiaDf.loc[bibliiaDf['book'] == book_name].chap.unique():
        #    c_min = bibliiaDf.loc[(bibliiaDf['book'] == book_name) & (bibliiaDf['chap'] == chap_name)].reset_index().p_id.min()
        #    c_max = bibliiaDf.loc[(bibliiaDf['book'] == book_name) & (bibliiaDf['chap'] == chap_name)].reset_index().p_id.max()
        #    print(f"{chap_name}: {c_min} - {c_max}")
#print(PartLineNumsDict)

O: 1 - 29147
N: 29148 - 37110


## Bible Phrase Dictionary

In [18]:
with open(bibFrazyTXT) as bibleFrazy: 
    bibleFrazyLines = bibleFrazy.readlines()

In [19]:
BibleFrazyDf = pd.DataFrame(bibleFrazyLines).rename(columns={0:'fraza'})
BibleFrazyDf.loc[:,'fraza'] = BibleFrazyDf.loc[:,'fraza'].apply(lambda x: x.strip().strip('\n').translate(str.maketrans('', '', string.punctuation)))
BibleFrazyDf = BibleFrazyDf.loc[BibleFrazyDf.fraza != '']

In [110]:
frazyList = BibleFrazyDf.loc[30189:].fraza.to_list()
FrazyDf = pd.DataFrame(data=frazyList).rename(columns={0:'fraza'})
FrazyDf.index.name = 'f_id'
FrazyDf

,fraza
f_id,
0,Агнец Божий непорочный как будто как будто сло...
1,Адамова голова
2,Адамово яблоко
3,Адамовы веки времена
4,Адамовы слёзы
...,...
535,Что посеешь то и пожнёшь
536,Юдоль плача печали скорби бед плачевная земная...
537,Явление Христа народу


In [111]:
%%capture
FrazyDf = nat_parse(FrazyDf, textCol='fraza')

In [112]:
FrazyDf['stopword'] = ~FrazyDf.loc[:, tokenCols[9:]].any(axis=1)
#FrazyDf = FrazyDf.set_index('p_id', 'token_id')
FrazyDf

/tmp/ipykernel_103847/3221199829.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only
  FrazyDf = FrazyDf.set_index('p_id', 'token_id')


,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
p_id,,,,,,,,,,,,,,,,,,,,
0,0,5,Агнец,1_1,1_3,nsubj,PROPN,агнец,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None,False
0,6,11,Божий,1_2,1_1,det,ADJ,божий,None,None,Nom,Pos,Masc,None,Sing,None,None,None,None,False
0,12,22,непорочный,1_3,1_0,root,VERB,непорочный,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Mid,False
0,23,26,как,1_4,1_10,mark,SCONJ,как,None,None,None,None,None,None,None,None,None,None,None,True
0,27,32,будто,1_5,1_6,fixed,PART,будто,None,None,None,None,None,None,None,None,None,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,4,7,аки,1_2,1_1,obj,NOUN,аки,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None,False
539,8,11,как,1_3,1_4,case,SCONJ,как,None,None,None,None,None,None,None,None,None,None,None,True
539,12,16,тать,1_4,1_2,acl,NOUN,тать,Inan,None,Nom,None,Masc,None,Sing,None,None,None,None,False


In [ ]:
for phrase_id in FrazyDf.p_id.unique().tolist():
   testPhrase = FrazyDf.loc[FrazyDf.p_id] 

In [118]:
pos_lists = []
for x in range(0,(FrazyDf.p_id.max()+1)):
    pos_lists.append(FrazyDf.loc[(FrazyDf.p_id == x) & (FrazyDf.stopword == False)].lemma.unique())

AttributeError: 'DataFrame' object has no attribute 'p_id'

In [115]:
#bibliiaDf.loc[(bibliiaDf.test == 'N')]
bibliiaDf.loc[(bibliiaDf.book == 'Lk') & (bibliiaDf.chap == 10) & (bibliiaDf.verse == '42')].index

Int64Index([31412], dtype='int64', name='p_id')

In [117]:
bibliiaDf.loc[31412].text

'а одно только нужно; Мария же избрала благую часть, которая не отнимется у нее. '

In [39]:
#phraseList = [x.tolist() for x in pos_lists]
for verse_id in range(31400,31450):
    print(f"verse_id: {verse_id}")
    testVerse = TokenDf.loc[(TokenDf.p_id == verse_id) & (TokenDf.stopword == False)].lemma.tolist()
    print(f"testVerse: {testVerse}")
    for phrase in pos_lists:
        print(f"phrase: {phrase}")
        i = 0
        for word in phrase:
            if word in testVerse:
                i+=1
        print(f"{verse_id} : {i} / {phrase.size}")

verse_id: 31400
testVerse: ['на', 'это', 'сказать', 'иисус', 'некоторый', 'человек', 'идти', 'из', 'иерусалим', 'в', 'иерихон', 'и', 'попасться', 'разбойник', 'который', 'снять', 'с', 'он', 'одежда', 'изранить', 'он', 'и', 'уйти', 'оставить', 'он', 'едва', 'живой']
phrase: ['агнец' 'божий' 'непорочный' 'как' 'будто' 'словно' 'точно']
31400 : 0 / 7
phrase: ['адамов' 'голова']
31400 : 0 / 2
phrase: ['адамов' 'яблоко']
31400 : 0 / 2
phrase: ['адамов' 'веко' 'время']
31400 : 0 / 3
phrase: ['адамов' 'слеза']
31400 : 0 / 2
phrase: ['ад' 'кромешный']
31400 : 0 / 2
phrase: ['ад' 'кромешный' 'на' 'душа' 'сердце' 'у' 'кто']
31400 : 1 / 7
phrase: ['аллилуйя' 'петь' 'кто']
31400 : 0 / 3
phrase: ['алчущие' 'и' 'жаждать']
31400 : 1 / 3
phrase: ['альфа' 'и' 'омега' 'что']
31400 : 1 / 4
phrase: ['аредов' 'веки' 'жить' 'прожить']
31400 : 0 / 4
phrase: ['а' 'судья' 'кто']
31400 : 0 / 3
phrase: ['бальзам' 'на' 'рана' 'для' 'душа' 'сердце' 'лить' 'проливать' 'пролить']
31400 : 1 / 9
phrase: ['башня' 'из' 

In [35]:
TokenDf.loc[TokenDf.p_id == 31400]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice
0,31400,0,2,На,1_1,1_2,case,ADP,на,None,None,None,None,None,None,None,None,None,None,None
1,31400,3,6,это,1_2,1_3,obl,PRON,это,Inan,None,Acc,None,Neut,None,Sing,None,None,None,None
2,31400,7,13,сказал,1_3,1_0,root,VERB,сказать,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Act
3,31400,14,19,Иисус,1_4,1_3,nsubj,PROPN,иисус,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None
4,31400,21,30,некоторый,1_6,1_7,det,DET,некоторый,None,None,Nom,None,Masc,None,Sing,None,None,None,None
5,31400,31,38,человек,1_7,1_8,nsubj,NOUN,человек,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None
6,31400,39,42,шел,1_8,1_4,parataxis,VERB,идти,None,Imp,None,None,Masc,Ind,Sing,None,Past,Fin,Act
7,31400,43,45,из,1_9,1_10,case,ADP,из,None,None,None,None,None,None,None,None,None,None,None
8,31400,46,56,Иерусалима,1_10,1_8,obl,PROPN,иерусалим,Inan,None,Gen,None,Masc,None,Sing,None,None,None,None
9,31400,57,58,в,1_11,1_12,case,ADP,в,None,None,None,None,None,None,None,None,None,None,None


In [32]:
FrazyDf.loc[FrazyDf.p_id == 31400].lemma.tolist()

[]

In [122]:
testPhrase = FrazyDf.loc[FrazyDf.p_id == 22].lemma.unique()

In [131]:
testVerse = TokenDf.loc[TokenDf.p_id == 31412].lemma.tolist()

In [130]:
i = 0
print(testPhrase.tolist())
print(testVerse)
for x in testPhrase.tolist():
    if x in testVerse:
        i+=1
print(f"i = {i}")
if i == testPhrase.size:
    print("phrase in verse")

['благой', 'часть', 'избрать']
['а', 'один', 'только', 'нужный', 'мария', 'же', 'избрать', 'благой', 'часть', 'который', 'не', 'отняться', 'у', 'она']
i = 3
phrase in verse


In [81]:
for x in pos_lists[20:25]:
    print(x)
    for y in TokenDf.p_id:
        #print(TokenDf.loc[TokenDf.p_id == y].lemma.unique())
        #if x in TokenDf.loc[TokenDf.p_id == y].lemma.unique():
            #print(f"{x} in {y}")

['бесплодный' 'смоковница']
['бить' 'колотить' 'ударять' 'себя' 'в' 'грудь']
['благой' 'часть' 'избрать']
['блаженный' 'кто' 'веровать' 'тепло' 'он' 'на' 'свет']
['блаженный' 'муж' 'иже' 'не' 'ида' 'на' 'совет' 'нечестивый']


In [15]:
FrazyAbbrsDf = BibleFrazyDf.iloc[:105].fraza.str.split(' – ', expand=True).rename(columns={0:"abbr", 1:"fraza"})
FrazyAbbrsDf.loc[:,'abbr'] = FrazyAbbrsDf.loc[:,'abbr'].apply(lambda x: x.strip())
FrazyAbbrsDf.loc[:,'fraza'] = FrazyAbbrsDf.loc[:,'fraza'].apply(lambda x: x.strip())
FrazyAbbrsDict = FrazyAbbrsDf.set_index('abbr').to_dict().get('fraza')
FrazyAbbrsDict

{'АиФ': 'Аргументы и факты',
 'Лит газета': '«Литературная газета»',
 'англ': 'английский',
 'межд': 'междометие',
 'ант': 'антоним антонимы',
 'местн п': 'местный падеж',
 'бел': 'белорусский',
 'МК': '«Московский комсомолец»',
 'библ': 'библейский',
 'болг': 'болгарский',
 'МН': '«Московские новости»',
 'бран': 'бранный',
 'мн ч': 'множественное число',
 'букв': 'буквально',
 'м р': 'мужской род',
 'в': 'век',
 'нареч': 'наречие',
 'вв': 'века',
 'нач': 'начало',
 'возм': 'возможно',
 'НГ': '«Независимая газета»',
 'высок': 'высокий',
 'негативн': 'негативный',
 'г': 'год',
 'неизм': 'неизменяемое',
 'гг': 'годы',
 'нем': 'немецкий',
 'глаг': 'глагол',
 'неодобр': 'неодобрительный',
 'гол': 'голландский',
 'нов': 'новый',
 'греч': 'греческий',
 'Нов изв': '«Новые известия»',
 'грубопрост': 'грубопросторечный',
 'нэ': 'новой эры',
 'дат п': 'дательный падеж',
 'обл': 'областное',
 'диал': 'диалектный',
 'одобр': 'одобрительный',
 'до нэ': 'до новой эры',
 'ок': 'около',
 'дргреч': 'др